In [60]:
import time
import torch
from tqdm import tqdm
import numpy as np
import logging
from hype.sn import Embedding 
from hype import train
from hype.graph import load_adjacency_matrix, load_edge_list, eval_reconstruction
from hype.rsgd import RiemannianSGD
from hype.Poincare import PoincareManifold
import sys
import json
import torch.multiprocessing as mp
from hype.graph_dataset import BatchedDataset

device = torch.device('cpu')
torch.set_default_tensor_type('torch.DoubleTensor')
#torch.set_default_tensor_type('torch.FloatTensor')


In [34]:
## parameters
opt_maxnorm = 500000; opt_com_n = 2; opt_debug = False
opt_manifold = "Poincare"; opt_dim = 2; opt_com_n = 1;
opt_negs = 50; opt_batchsize = 10; opt_eval_each = 20;
opt_sparse = True; opt_ndproc = 4;  opt_burnin = 20;
opt_dampening = 0.75; opt_neg_multiplier = 1.0; 
opt_burnin_multiplier = 0.01; opt_lr = 0.3

### Initializing logging and data loading

In [18]:
log_level = logging.DEBUG if opt_debug else logging.INFO
log = logging.getLogger('Poincare')
logging.basicConfig(level=log_level, format='%(message)s', stream=sys.stdout)
log.info('Using edge list dataloader')
idx, objects, weights = load_edge_list("wordnet/mammal_closure.csv", False) 

Using edge list dataloader


### Initializing model

In [46]:
def init_model(manifold, idx, objects, weights, sparse=True):
    conf = []
    model_name = '%s_dim%d%com_n'
    mname = model_name % (opt_manifold, opt_dim, opt_com_n)
    data = BatchedDataset(idx, objects, weights, opt_negs, opt_batchsize,
        opt_ndproc, opt_burnin > 0, opt_dampening)
    model = Embedding(len(data.objects), opt_dim, manifold, sparse=sparse, com_n=opt_com_n)
    data.objects = objects
    return model, data, mname, conf

def adj_matrix(data):
  adj = {}
  for inputs, _ in data:
    for row in inputs:
        x = row[0].item(); y = row[1].item()
        if x in adj:
            adj[x].add(y)
        else:
            adj[x] = {y}
  return adj

In [49]:
manifold = PoincareManifold(
    debug=opt_debug, max_norm=opt_maxnorm, com_n=opt_com_n)
model, data, model_name, conf = init_model(
    manifold, idx, objects, weights, sparse=opt_sparse)
model = model.to(device)
print('the total dimension', model.lt.weight.data.size(-1), 'com_n', opt_com_n)
# set burnin parameters
data.neg_multiplier = opt_neg_multiplier
train._lr_multiplier = opt_burnin_multiplier
# setup optimizer
optimizer = RiemannianSGD(model.optim_params(manifold), lr= opt_lr)
# get adjacency matrix
adj = adj_matrix(data) 

the total dimension 2 com_n 1


In [61]:
tqdm(data)

  0%|          | 0/654 [00:00<?, ?it/s]

  0%|          | 0/654 [00:00<?, ?it/s]

### Training

In [51]:
def train(device, model, data, optimizer, opt, progress=False ):
  loader_iter = tqdm(data) if progress else data
  
  return
  
  

In [50]:
device, model, data, optimizer,


(device(type='cpu'),
 Embedding(
   (lt): Embedding(1180, 2, sparse=True)
 ),
 RiemannianSGD (
 Parameter Group 0
     expm: <bound method EuclideanManifold.expm of <hype.Poincare.PoincareManifold object at 0x7fc9a0d5aa48>>
     logm: <bound method EuclideanManifold.logm of <hype.Poincare.PoincareManifold object at 0x7fc9a0d5aa48>>
     lr: 0.3
     ptransp: <bound method EuclideanManifold.ptransp of <hype.Poincare.PoincareManifold object at 0x7fc9a0d5aa48>>
     rgrad: <bound method PoincareManifold.rgrad of <hype.Poincare.PoincareManifold object at 0x7fc9a0d5aa48>>
 ))